In [1]:
sc

In [159]:
ls = sc.textFile('data/coupon150720.csv', 100) #indico con 100 el número de particiones donde almacenar los datos

In [21]:
ls.getNumPartitions()

100

In [5]:
ls.take(5)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0',
 u'79062005698500,2,AUH,CDG,9W,9W,84.34,USD,1,H,H,6120,150905,OK,IAF0',
 u'79062005924069,1,CJB,MAA,9W,9W,60.0,USD,1,H,H,2768,150721,OK,IAA0',
 u'79065668570385,1,DEL,DXB,9W,9W,160.63,USD,2,S,S,0546,150804,OK,INA0',
 u'79065668737021,1,AUH,IXE,9W,9W,152.46,USD,1,V,V,0501,150803,OK,INA0']

In [20]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [13]:
ls.count()

1232662

In [16]:
ls.cache()

MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:-2

In [18]:
ls.count()

1232662

In [17]:
#Controlar la cache de los datos
ls.persist()

1232662

¿Cuántos pasajeros han volado con Iberia?

In [39]:
ls.map(lambda x: x.split(',')).take(5)

[[u'79062005698500',
  u'1',
  u'MAA',
  u'AUH',
  u'9W',
  u'9W',
  u'56.79',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'0526',
  u'150904',
  u'OK',
  u'IAF0'],
 [u'79062005698500',
  u'2',
  u'AUH',
  u'CDG',
  u'9W',
  u'9W',
  u'84.34',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'6120',
  u'150905',
  u'OK',
  u'IAF0'],
 [u'79062005924069',
  u'1',
  u'CJB',
  u'MAA',
  u'9W',
  u'9W',
  u'60.0',
  u'USD',
  u'1',
  u'H',
  u'H',
  u'2768',
  u'150721',
  u'OK',
  u'IAA0'],
 [u'79065668570385',
  u'1',
  u'DEL',
  u'DXB',
  u'9W',
  u'9W',
  u'160.63',
  u'USD',
  u'2',
  u'S',
  u'S',
  u'0546',
  u'150804',
  u'OK',
  u'INA0'],
 [u'79065668737021',
  u'1',
  u'AUH',
  u'IXE',
  u'9W',
  u'9W',
  u'152.46',
  u'USD',
  u'1',
  u'V',
  u'V',
  u'0501',
  u'150803',
  u'OK',
  u'INA0']]

In [41]:
#mi solución inicial
aerolineas = ls.map(lambda x: x.split(',')).filter(lambda x: x[5]=='IB').count()

26158

In [42]:
# solución más óptima de rendimiento
aerolineas = ls.map(lambda x: x.split(',')[5]).filter(lambda x: x=='IB').count()

In [43]:
aerolineas

26158

In [44]:
def getAerolineaDinero(l):
    elems = l.split(",")
    aerolinea = elems[5]
    dinero = float(elems[6])
    return (aerolinea, dinero)


In [45]:
getAerolineaDinero(ls.first())

(u'9W', 56.79)

In [46]:
dineros = ls.map(getAerolineaDinero)

In [48]:
dineros.take(3)

[(u'9W', 56.79), (u'9W', 84.34), (u'9W', 60.0)]

In [53]:
dineros.filter(lambda x: x[0]=='IB').take(3)

[(u'IB', 313.99), (u'IB', 865.32), (u'IB', 16.58)]

In [54]:
dinerosIB = dineros.filter(lambda x: x[0]=='IB')

In [64]:
cantidades = dinerosIB.map(lambda x: x[1])

In [67]:
cantidades.take(3)

[313.99, 865.32, 16.58]

In [66]:
cantidades.sum()

2828044.420000028

In [65]:
cantidades.stats()

(count: 26158, mean: 108.113939139, stdev: 168.011485542, max: 5239.93, min: 0.0)

        #### ¿Cuál es la ruta con el billete más caro?

In [68]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [72]:
def getRutaDinero(l):
    e = l.split(',')
    origen = e[2]
    destino = e[3]
    dinero = float(e[6])
    ruta = origen + '-' + destino
    return (ruta, dinero)

In [73]:
getRutaDinero(ls.first())

(u'MAA-AUH', 56.79)

In [74]:
rutas = ls.map(getRutaDinero)

In [75]:
rutas.take(3)

[(u'MAA-AUH', 56.79), (u'AUH-CDG', 84.34), (u'CJB-MAA', 60.0)]

In [77]:
#ordenamos desc
rutas.sortBy(lambda x: x[1], False).take(5)

[(u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'SYD-CGK', 6355194.0),
 (u'CDG-IAD', 637313.52),
 (u'IAD-CDG', 637313.52)]

In [78]:
# como hay importes incorrectos (6355194 un billete) vamos a filtrar
rutas.filter(lambda x: x[1] < 7500).sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'MIA-CCS', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0)]

In [79]:
filtrado = rutas.filter(lambda x: x[1] < 7500)

In [81]:
filtrado.cache()

PythonRDD[87] at RDD at PythonRDD.scala:43

In [82]:
filtrado.groupByKey().take(3)

[(u'TRV-MCT', <pyspark.resultiterable.ResultIterable at 0x7f6dd3282090>),
 (u'IAD-SFO', <pyspark.resultiterable.ResultIterable at 0x7f6dd32826d0>),
 (u'YVR-TPE', <pyspark.resultiterable.ResultIterable at 0x7f6dd3282490>)]

In [84]:
#precio máximo por ruta
filtrado.groupByKey().mapValues(lambda vs: max(vs)).take(3)

[(u'TRV-MCT', 220.44), (u'IAD-SFO', 1578.63), (u'YVR-TPE', 877.71)]

In [85]:
maximos = filtrado.groupByKey().mapValues(lambda vs: max(vs))

In [86]:
maximos.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MIA', 7490.33),
 (u'CCS-MIA', 7415.0),
 (u'MIA-CCS', 7415.0),
 (u'RUH-LAX', 7401.94),
 (u'DXB-MAA', 7343.27)]

#### Ruta con el valor medio más alto

In [89]:
medios = filtrado.groupByKey().mapValues(lambda vs: sum(vs) / len(vs))

In [90]:
medios.take(5)

[(u'TRV-MCT', 124.93499999999999),
 (u'IAD-SFO', 222.7460256410257),
 (u'YVR-TPE', 401.2866666666668),
 (u'LED-HEL', 40.0827659574468),
 (u'OMA-ATL', 103.34181818181818)]

In [91]:
medios.sortBy(lambda x: x[1], False).take(5)

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

In [92]:
# Todo junto
filtrado.groupByKey().mapValues(lambda vs: sum(vs) / len(vs)) \
.sortBy(lambda x: x[1], False) \
.take(5)

[(u'BLA-MEC', 5164.355),
 (u'MEC-BLA', 5164.355),
 (u'CCS-MIA', 4350.516296296297),
 (u'MIA-CCS', 4186.95),
 (u'NRT-PPT', 3751.5499999999997)]

##### Vamos a agrupar por algún criterio más: booking class, tarifa... con el fin de ver los outliers por categoría


In [93]:
ls.take(1)

[u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0']

In [121]:
#Campos: 
def getRutaALDineroClase(l):
    e = l.split(',')
    ruta = e[2] + '-' + e[3]
    al = e[4]
    dinero = float(e[6])
    clase = e[9]
    return ((ruta, al, clase), dinero)

In [97]:
getRutaALDineroClase(ls.first())

((u'MAA-AUH', u'9W', u'H'), 56.79)

In [98]:
# valor limite para las variables
import math


In [135]:
#vamos a quitar los valores altos (por encima de dos veces la desviación estandart)
def valorLimite(vs):
    logs = [math.log(v+1) for v in vs] #sumamos 1 ya que hay valores que son 0
    # vs.map(lambda x : log(x))
    media = sum(logs)/len(logs)
    desv = [(x-media)**2 for x in logs]
    var = sum(desv)/len(desv)
    desvStd = math.sqrt(var)
    return math.exp(media + 2*desvStd) - 1 # lo exponenciamos para poder devolverlo en la medida inicial y le restamos el 1 anterior

In [134]:
valorLimite([12, 3, 4, 145]) # los datos de muestra no son normales por lo que aquí no aplicaría

240.70561554377625

In [124]:
ls.map(getRutaALDineroClase).take(3)
rutas = ls.map(getRutaALDineroClase)

In [125]:
rutas.groupByKey().take(2)

[((u'LAX-SJC', u'AA', u'E'),
  <pyspark.resultiterable.ResultIterable at 0x7f6dd3282ed0>),
 ((u'PAD-PMI', u'AB', u'X'),
  <pyspark.resultiterable.ResultIterable at 0x7f6dd3282210>)]

In [126]:
def filtraCuponesExtremos(vs):
    vl = valorLimite(vs)
    f = [v for v in vs if v < vl]
    return f

In [136]:
rutas.groupByKey().mapValues(filtraCuponesExtremos).take(2)

[((u'LAX-SJC', u'AA', u'E'), []), ((u'PAD-PMI', u'AB', u'X'), [])]

In [147]:
grupos = rutas.groupByKey().mapValues(filtraCuponesExtremos)

In [139]:
#Falla por que el filtro es muy estricto
# grupos.mapValues(lambda vs: sum(vs) / len(vs)).take(5)

In [140]:
def mediaGrupo(vs):
    if(len(vs) == 0):
        return 0.0
    else:
        return (sum(vs)/len(vs))

In [145]:
rutas.groupByKey().mapValues(mediaGrupo).take(5)
#vemos que hay grupos con 0 que son los que hacían que fallara



In [148]:
#vamos a hacerlo por grupo
grupos.mapValues(mediaGrupo).sortBy(lambda x: x[1], False).take(5)

[((u'SYD-CGK', u'GA', u'G'), 6355194.0),
 ((u'COO-CDG', u'AF', u'J'), 538599.2),
 ((u'CDG-COO', u'AF', u'J'), 179628.3533333333),
 ((u'DKR-NBO', u'KQ', u'T'), 58253.593333333345),
 ((u'TSN-ICN', u'KE', u'Z'), 38655.45)]

In [152]:
# vamos a filtrar todos los grupos pequeños, ya que en este caso no tenemos una distribución normal
# grupos.filter(lambda x: len(x[1])> 10).mapValues(filtraCuponesExtremos).take(2)
r = grupos.filter(lambda x: len(x[1])> 10).mapValues(filtraCuponesExtremos).mapValues(mediaGrupo)

In [153]:
# Billetes medios más caros
r.sortBy(lambda x: x[1], False).take(5)

[((u'MIA-CCS', u'S3', u'S'), 7826.25),
 ((u'CCS-MIA', u'S3', u'B'), 5550.0),
 ((u'CCS-MAD', u'V0', u'Y'), 5418.098666666667),
 ((u'LHR-DOH', u'QR', u'F'), 4275.98923076923),
 ((u'CCS-PTY', u'S3', u'S'), 3583.0153846153844)]

#### Cuantas aerolíneas diferentes hay?

In [160]:
ls.first()

u'79062005698500,1,MAA,AUH,9W,9W,56.79,USD,1,H,H,0526,150904,OK,IAF0'

In [ ]:
ls.map(lambda x: x.split(",")[4]).distinct().count()

In [164]:
ls.map(lambda x: x.split(",")[7]).distinct().collect()

[u'USD']

In [169]:
#me genera tantos ficheros como particiones
ls.map(lambda x: x.split(",")[4]).distinct().saveAsTextFile("data/results.csv")

Py4JJavaError: An error occurred while calling o1748.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/dsc/Repositories/Master-in-Data-Science/spark/data/results.csv already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:132)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:147)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:108)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:310)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1065)
	at org.apache.spark.rdd.PairRDDFuncti

In [167]:
ls.map(lambda x: x.split(",")[4]).distinct().take(5)

[u'USD']